In [ ]:
#pull in inventories

In [ ]:
import os, datetime
import csv
import sys
import shutil
from openpyxl import load_workbook
import pandas as pd
from io import BytesIO
import numpy as np
import subprocess
from scipy import stats
from ccf.box import LifespanBox
from ccf.config import LoadSettings
from ccf.redcap import RedcapTable 

pathinvent='/home/petra/CCF_QC/PreRelease/'

In [ ]:
box=LifespanBox(cache='./')
verbose = True
snapshotdate = datetime.datetime.today().strftime('%m_%d_%Y')

DI='HCD_Inventory_Restricted_2022-02-04.csv'
Dpathout="./prepped/hcd" 
Deventlist=['visit_1_arm_1','visit_2_arm_1','visit_3_arm_1','visit_arm_1']
inventoryD=pd.read_csv(pathinvent+DI)
inventoryD=inventoryD.loc[(inventoryD.nda_age.isnull()==False) & (inventoryD.redcap_event_name.isin(Deventlist)) ]
inventoryD.nda_interview_date=pd.to_datetime(inventoryD.nda_interview_date).dt.strftime('%m/%d/%Y')

AI='HCA_Inventory_Restricted_2022-02-04.csv'
Apathout="./prepped/hca" 
Aeventlist=['visit_1_arm_1','visit_2_arm_1','visit_3_arm_1','visit_arm_1']
inventoryA=pd.read_csv(pathinvent+AI)
inventoryA=inventoryA.loc[(inventoryA.nda_age.isnull()==False) & (inventoryA.redcap_event_name.isin(Aeventlist))]
inventoryA.nda_interview_date=pd.to_datetime(inventoryA.nda_interview_date).dt.strftime('%m/%d/%Y')

In [ ]:
inventoryD=inventoryD[['pseudo_guid','nda_age', 'nda_interview_date','subject', 'M/F','redcap_event']]#,'parent_at_V1',,'HCAid', 'HCDid','site', 'race', 'ethnic_group']]
inventoryA=inventoryA[['pseudo_guid','nda_age', 'nda_interview_date','subject','M/F','redcap_event']]#'HCAid', 'HCDid','site', 'race', 'ethnic_group', 


In [ ]:
iD=inventoryD
for filename in os.listdir(Dpathout):
    a=pd.read_csv(os.path.join(Dpathout, filename),header=1)
    struct=filename[0:-4]
    print(struct)
    a[struct]=1
    a=a[['src_subject_id','interview_date',struct]]
    iD=pd.merge(iD,a,left_on=['subject','nda_interview_date'],right_on=['src_subject_id','interview_date'],how='left')
    iD[struct] = pd.to_numeric(iD[struct], errors='coerce').astype('Int64')
    iD.loc[iD[struct].isnull()==True,struct]=0
    iD=iD.drop(columns=['src_subject_id','interview_date'])



In [ ]:
iD=iD.rename(columns={'pseudo_guid':'subjectkey','nda_age':'interview_age','nda_interview_date':'interview_date','M/F':'sex','redcap_event':'visit'})#.drop(columns=['HCDid','HCAid','site','race','ethnic_group','parent_at_V1'])
iD.to_csv('test.csv')   


In [ ]:
iD=iD.rename(columns={'subject':'src_subject_id','pseudo_guid':'subjectkey','nda_age':'interview_age','nda_interview_date':'interview_date','M/F':'sex','redcap_event':'visit'})#.drop(columns=['HCDid','HCAid','site','race','ethnic_group'])
iD.head()
cols=list(iD.columns)
newlistb = ["Any data in "+x+"? 1=yes; 0=no" for x in cols if ("01" in x) or ("02" in x)]
newlista = [x for x in cols if "01" not in x] #checking
newlistamap = ['Pseudo-Guid', 'age in months', 'RedCap event registration date when copied to IntraDB (rounded down to nearest Quarter)','subject','sex at birth','visit/event']
secondheader=newlistamap+newlistb
#secondheader
sechead=','.join(secondheader)
sechead


In [ ]:
#Write out structure for validation
filePath='./CompletenessHCD.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write(sechead+"\n")
    iD.to_csv(f,index=False)
    
    
iD.to_csv('CompletenessHCD.csv',index=False)   


In [ ]:
iA=inventoryA
for filename in os.listdir(Apathout):
    a=pd.read_csv(os.path.join(Apathout, filename),header=1)
    struct=filename[0:-4]
    print(struct)
    a[struct]=1
    a=a[['src_subject_id','interview_date',struct]]
    iA=pd.merge(iA,a,left_on=['subject','nda_interview_date'],right_on=['src_subject_id','interview_date'],how='left')
    iA[struct] = pd.to_numeric(iA[struct], errors='coerce').astype('Int64')
    iA.loc[iA[struct].isnull()==True,struct]=0
    iA=iA.drop(columns=['src_subject_id','interview_date'])




In [ ]:
iA=iA.rename(columns={'subject':'src_subject_id','pseudo_guid':'subjectkey','nda_age':'interview_age','nda_interview_date':'interview_date','M/F':'sex','redcap_event':'visit'})#.drop(columns=['HCDid','HCAid','site','race','ethnic_group'])
iA.head()
cols=list(iA.columns)
newlistb = ["Any data in "+x+"? 1=yes; 0=no" for x in cols if "01" in x]
newlista = [x for x in cols if "01" not in x] #checking
newlistamap = ['Pseudo-Guid', 'age in months', 'RedCap event registration date when copied to IntraDB (rounded down to nearest Quarter)','subject','sex at birth','visit/event']
secondheader=newlistamap+newlistb
#secondheader
sechead=','.join(secondheader)


In [ ]:
#Write out structure for validation
filePath='./CompletenessHCA.csv'
if os.path.exists(filePath):
    os.remove(filePath)
else:
    print("Can not delete the file as it doesn't exists")

with open(filePath,'a') as f:
    f.write(sechead+"\n")
    iA.to_csv(f,index=False)
iA.to_csv('CompletenessHCA.csv',index=False)   
